In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from data_preprocessing import get_input_RNN

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Activation, Bidirectional
from keras.layers.embeddings import Embedding

from tensorflow.keras import regularizers
from io import BytesIO

In [69]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units, l2_kernel, l2_recurrent, l2_activity):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim,
                                trainable = True, mask_zero = True)(X_input)
    
    X = LSTM(units = lstm_units, return_sequences = False,
            kernel_regularizer = regularizers.l2(l2_kernel),
            recurrent_regularizer = regularizers.l2(l2_recurrent),
            activity_regularizer = regularizers.l2(l2_activity))(embedding_layer)
    
    X = Dense(units = 1)(X)
    X = Activation('sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [70]:
#model = create_model(10000, 500, 100, lstm_units = 128, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0.01)
#model.summary()

In [71]:
def grid_search(X, dataset, epochs, models):
    
    Y = dataset.label.values
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)

    mean_dev_accuracy = np.zeros(np.size(models))
    
    i = 0
    for model in models:
    
        iterations = 5
        dev_accuracy_shuffle_split = np.zeros(iterations)
        shuffle = ShuffleSplit(n_splits = iterations, test_size = 0.2)
        
        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile model
        w = BytesIO()
        model.save_weights(w) # Save initial weights
        
        j = 0
        for train, dev in shuffle.split(X_train, y_train):
            Xtrain = X_train[train]
            Ytrain = y_train[train]
            Xdev = X_train[dev]
            Ydev = y_train[dev]

            model.fit(Xtrain, Ytrain, epochs = epochs, batch_size = 32, shuffle = True) # Fit model

            loss, accuracy_val = model.evaluate(Xdev, Ydev) # Validate model
            dev_accuracy_shuffle_split[j] = accuracy_val

            model.load_weights(w) # Restore initial weights
            
            j += 1
              
        mean_dev_accuracy[i] = round(np.mean(dev_accuracy_shuffle_split), 3)
        std = round(np.std(dev_accuracy_shuffle_split), 3)
        print('Model ' + str(i) +' --> dev_acc: ' + str(mean_dev_accuracy[i]) + ' +- ' + str(std))
        i += 1
    
    # test
    best_model_index = np.argmax(mean_dev_accuracy)
    best_model = models[best_model_index]
    best_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Compile best model
    best_model.fit(X_train, y_train, epochs = epochs, batch_size = 32, shuffle = True) # Fit best model
    y_pred = best_model.predict(X_test) # Test best model
    y_pred = y_pred > 0.5 # Sigmoid activation function
    accuracy_test = accuracy_score(y_test, y_pred)
    
    return best_model_index, round(accuracy_test, 3)

In [72]:
dataset = pd.read_csv('../data/Merged/spanish_dataset.csv')
dataset

,text,label
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",1
1,"La palabra ""haiga"", aceptada por la RAE La Rea...",1
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,1
3,UNAM capacitará a maestros para aprobar prueba...,0
4,Alerta: pretenden aprobar libros escolares con...,1
5,Un paso más cerca de hacer los exámenes 'onlin...,0
6,UNAM REALIZARÁ PRUEBAS ANTIDOPING A ESTUDIANTE...,1
7,Niño de *NUMBER* años se prepara para entrar a...,0
8,*NUMBER* palabras que creíamos inaceptables y ...,0
9,LIMITARÁN EL TIEMPO DE EGRESO EN FILOSOFÍA Y S...,1


#### Models

In [73]:
vocabulary_length = 10000
max_length_sequence = 500
emb_dim = 100
language = 'spanish'
epochs = 7

In [74]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 128, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 32, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1)
model_4 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 64, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1)
model_5 = create_model(vocabulary_length, max_length_sequence, emb_dim, lstm_units = 128, l2_kernel = 1, l2_recurrent = 1, l2_activity = 0.1)

models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)
models.append(model_4)
models.append(model_5)

In [75]:
X, df = get_input_RNN(dataset, stemming = False, remove_stopwords = True, 
                      vocabulary_length = vocabulary_length, max_length_sequence = max_length_sequence, language = language)
print(X.shape)

(2571, 500)


In [76]:
best_model_index, test_acc = grid_search(X, df, epochs, models)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/7
1644/1644 [==============================] - 21s 13ms/step - loss: 1.6246 - accuracy: 0.5572
Epoch 2/7
1644/1644 [==============================] - 21s 13ms/step - loss: 0.9466 - accuracy: 0.8151
Epoch 3/7
1644/1644 [==============================] - 24s 15ms/step - loss: 0.4523 - accuracy: 0.9215
Epoch 4/7
1644/1644 [==============================] - 35s 21ms/step - loss: 0.2024 - accuracy: 0.9793
Epoch 5/7
1644/1644 [==============================] - 50s 30ms/step - loss: 0.1522 - accuracy: 0.9891
Epoch 6/7
1644/1644 [==============================] - 40s 24ms/step - loss: 0.0951 - accuracy: 0.9976
Epoch 7/7
412/412 [==============================] - 3s 8ms/step
Epoch 1/7
1644/1644 [==============================] - 30s 19ms/step - loss: 1.5104 - accuracy: 0.5213
Epoch 2/7
1644/1644 [==============================] - 31s 19ms/step - loss: 0.8818 - accuracy: 0.6849
Epoch 3/7
1644/1644 [==============================] - 30s 18ms/step - loss: 0.7615 - accuracy: 0.8583
Epoch 4/

1644/1644 [==============================] - 98s 59ms/step - loss: 0.1864 - accuracy: 0.9860
Epoch 5/7
1644/1644 [==============================] - 68s 41ms/step - loss: 0.1500 - accuracy: 0.9951
Epoch 6/7
1644/1644 [==============================] - 72s 44ms/step - loss: 0.0862 - accuracy: 0.9988
Epoch 7/7
412/412 [==============================] - 6s 14ms/step
Epoch 1/7
1644/1644 [==============================] - 67s 41ms/step - loss: 1.8512 - accuracy: 0.5547
Epoch 2/7
1644/1644 [==============================] - 76s 46ms/step - loss: 0.8433 - accuracy: 0.7646
Epoch 3/7
1644/1644 [==============================] - 72s 44ms/step - loss: 0.7594 - accuracy: 0.8443
Epoch 4/7
1644/1644 [==============================] - 66s 40ms/step - loss: 0.6211 - accuracy: 0.9039
Epoch 5/7
1644/1644 [==============================] - 71s 43ms/step - loss: 0.3250 - accuracy: 0.9373
Epoch 6/7
1644/1644 [==============================] - 71s 43ms/step - loss: 0.1748 - accuracy: 0.9903
Epoch 7/7
412/412

412/412 [==============================] - 3s 7ms/step
Epoch 1/7
1644/1644 [==============================] - 23s 14ms/step - loss: 86.0888 - accuracy: 0.6107
Epoch 2/7
1644/1644 [==============================] - 24s 15ms/step - loss: 6.1190 - accuracy: 0.7962
Epoch 3/7
1644/1644 [==============================] - 26s 16ms/step - loss: 1.2572 - accuracy: 0.7731
Epoch 4/7
1644/1644 [==============================] - 25s 15ms/step - loss: 0.7893 - accuracy: 0.6606
Epoch 5/7
1644/1644 [==============================] - 25s 15ms/step - loss: 0.7127 - accuracy: 0.7652
Epoch 6/7
1644/1644 [==============================] - 24s 15ms/step - loss: 0.6927 - accuracy: 0.7987
Epoch 7/7
412/412 [==============================] - 2s 4ms/step
Epoch 1/7
1644/1644 [==============================] - 23s 14ms/step - loss: 77.4391 - accuracy: 0.6144
Epoch 2/7
1644/1644 [==============================] - 23s 14ms/step - loss: 3.6520 - accuracy: 0.6782
Epoch 3/7
1644/1644 [==============================] -